## Below is just the code how to do Incremental Learning

#### We are just providing a simple code to create a 1D convolutional neural network and do a staggered training. We have given example of how simple function of keras can help you to achieve your job to restart the training where you have left last time due to any issues like network disruption or instance termination etc

Let's define the network and it's parameter first

In [ ]:
LR = 0.001
optimizer = RMSprop(lr=LR)
CONV = [
    {'filters':512, 'kernel':8, 'strides':1, 'padding':'same', 'reg': 0.0, 'pool':2},
    {'filters':512, 'kernel':8, 'strides':1, 'padding':'same', 'reg': 0.0, 'pool':2},
    {'filters':512, 'kernel':8, 'strides':1, 'padding':'same', 'reg': 0.0, 'pool':''}
]

In [ ]:
inputlayer = Input(shape=(400, 26))

network = inputlayer

for C in CONV:

    # conv layer
    network = Conv1D(filters=C['filters'], kernel_size=C['kernel'], \
                     strides=C['strides'], padding=C['padding'], activation='relu', \
                     kernel_regularizer=regularizers.l2(C['reg']))(network)

    if type(C['pool']) != int:
        continue

    # pooling layer
    network = MaxPooling1D(C['pool'])(network)

# fully connected --------------
# ------------------------------
network = Flatten()(network)
network = Dense(1024, activation='relu')(network)
network = Dropout(0)(network)
ypred = Dense(12, activation='softmax')(network)

In [ ]:
model = Model(inputs=inputlayer, outputs=ypred)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

Once the model is defined it's time to define check pints where we are putting a criteria if the validation accuracy is not improving for 10 epochs (Iteration) then the training would stop.


We have a function that saves the model weights after each epoch if the validation loss decreased

In [ ]:

estopping = EarlyStopping(monitor='val_acc', patience=10)
checkpoint = ModelCheckpoint(filepath='model_0207', save_best_only=True) #saves the model weights after each epoch if the validation loss decreased

Now let's start training without worring about connection termination or anyother issue:

In [ ]:
for i in range(10):
    
    try:
        hist = model.fit(X_train,
                         y_train,
                         validation_data=(X_test, y_test),
                         epochs=100,
                         batch_size=256,
                         shuffle=False,
                         verbose=2,
                         callbacks=[checkpoint, estopping])
 
    except KeyboardInterrupt:    
        print("training terminated by user")
        
    model.save('/content/MyModelSummary/ModelSummary/model.h5') # creates a HDF5 file 'my_model.h5'

create a directory to save model summary in the mounted directory,this is where we save all the model specifications

* the architecture of the model, allowing to re-create the model

* the weights of the model
* the training configuration (loss, optimizer)
* the state of the optimizer, allowing to resume training exactly where you left off.

      ---->model.save('/content/MyModelSummary/ModelSummary/model.h5')  # creates a HDF5 file 'my_model.h5'

#### Now next time you want to start training you have already trained model with best weights till that time along with all other parameters

In [ ]:
#From the next time,just load the model with the below lines and continue with your model execution
from keras.models import load_model

# returns a compiled model,identical to the previous run
model_2 = load_model('/content/MyModelSummary/ModelSummary/model.h5')
model_2.summary()

In [ ]:
#We are running the model with previous check point 
try:
    hist = model_2.fit(X_train,
                     y_train,
                     validation_data=(X_test, y_test),
                     epochs=100,
                     batch_size=256,
                     shuffle=False,
                     verbose=2,
                     callbacks=[checkpoint, estopping, tensorboard])
 
except KeyboardInterrupt:    
    print("training terminated by user")